# Vehicle Search Agent API Testing

Interactive notebook for testing the API endpoints.

**Prerequisites:** Make sure the API server is running:
```bash
python api/server.py
api documentation: http://localhost:8000/docs
```

In [2]:
import requests
import json
from pprint import pprint

BASE_URL = "http://localhost:8000"

## 1. Health Check

In [3]:
response = requests.get(f"{BASE_URL}/")
print(f"Status: {response.status_code}")
pprint(response.json())

Status: 200
{'service': 'IDSS API', 'status': 'online', 'version': '1.0.0'}


## 2. Start a Conversation

In [4]:
# Send first message
response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "I want a Jeep around $30k in Colorado"
    }
)

data = response.json()
session_id = data.get('session_id')

print(f"Session ID: {session_id}")
print(f"Mode: {data.get('mode')}")
print(f"\nAgent Response:")
print(data.get('response'))
print(f"\nVehicles Found: {len(data.get('vehicles', []))}")

Session ID: a23406a0-f591-42c4-8b88-56e056acdd50
Mode: None

Agent Response:
Hey there! I see you're on the hunt for a Jeep in Colorado within a $28-32k budget. There are a couple of interesting options here: one is priced at $28,000 with 23,815 miles, and another at $31,995 with only 15,857 miles. Both are great choices, with the first being a budget-friendly pick and the second offering lower mileage. Are you looking for something more city-friendly or off-road capable? And how important is mileage for you in your decision?

Vehicles Found: 20


## 3. View Filters

In [5]:
print("Current Filters:")
pprint(data.get('filters'))

print("\nInferred Preferences:")
pprint(data.get('preferences'))

Current Filters:
{'make': 'Jeep', 'price': '28000-32000', 'state': 'CO'}

Inferred Preferences:
{'brand_affinity': ['Jeep'], 'budget_sensitivity': 'moderate'}


## 4. View Vehicle Listings

In [18]:
vehicles = data.get('vehicles', [])
for i, vehicle in enumerate(vehicles[:5], 1):  # Show top 5
    v = vehicle.get('vehicle', {})
    listing = vehicle.get('retailListing', {})
    history = vehicle.get('history', {})
    photos = vehicle.get('photos')

    print(f"\n{'='*80}")
    print(f"#{i}. {v.get('year')} {v.get('make')} {v.get('model')} {v.get('trim', '')}")
    print(f"{'='*80}")

    # Price and basic info
    print(f"💰 Price:        ${listing.get('price', 0):,}")
    print(f"📏 Mileage:      {listing.get('miles', 0):,} miles")
    print(f"📍 Location:     {listing.get('city')}, {listing.get('state')} {listing.get('zip')}")
    print(f"🏢 Dealer:       {listing.get('dealer')}")

    # Vehicle details
    print(f"\n🚗 Vehicle Details:")
    print(f"   • Vehicle Details Page: {listing.get('vdp', 'N/A')}")
    print(f"   • Body Style:    {v.get('bodyStyle', 'N/A')}")
    print(f"   • Drivetrain:    {v.get('drivetrain', 'N/A')}")
    print(f"   • Engine:        {v.get('engine', 'N/A')}")
    print(f"   • Transmission:  {v.get('transmission', 'N/A')}")
    print(f"   • Exterior:      {v.get('exteriorColor', 'N/A')}")
    print(f"   • Interior:      {v.get('interiorColor', 'N/A')}")
    print(f"   • Doors:         {v.get('doors', 'N/A')}")
    print(f"   • Seats:         {v.get('seats', 'N/A')}")

    # History
    print(f"\n📋 Vehicle History:")
    print(f"   • Accidents:     {'Yes (' + str(history.get('accidentCount', 0)) + ')' if
history.get('accidents') else 'No'}")
    print(f"   • Owners:        {history.get('ownerCount', 'N/A')}")
    print(f"   • Usage:         {history.get('usageType', 'N/A')}")

    # Photos
    if photos and photos.get('retail'):
        photo_urls = photos.get('retail', [])
        print(f"\n📸 Photos:       {len(photo_urls)} available (cached)")
        print(f"   • First photo:   {photo_urls[0][:60]}...")
    else:
        print(f"\n📸 Photos:       No photos available")

    # Links
    print(f"\n🔗 Links:")
    print(f"   • VIN:           {v.get('vin')}")
    print(f"   • Listing:       {listing.get('vdp', 'N/A')[:70]}...")
    if listing.get('carfaxUrl'):
        print(f"   • CarFax:        {listing.get('carfaxUrl')[:70]}...")

print(f"\n{'='*80}")
print(f"Total vehicles: {len(vehicles)}")


#1. 2017 Jeep Wrangler Sahara
💰 Price:        $28,990
📏 Mileage:      18,353 miles
📍 Location:     Colorado Springs, CO 80906
🏢 Dealer:       Carvana

🚗 Vehicle Details:
   • Vehicle Details Page: https://www.carvana.com/vehicle/3898305
   • Body Style:    Convertible SUV
   • Drivetrain:    4WD
   • Engine:        6Cyl Regular Unleaded
   • Transmission:  Manual
   • Exterior:      Other
   • Interior:      Black
   • Doors:         2
   • Seats:         4

📋 Vehicle History:
   • Accidents:     No
   • Owners:        1
   • Usage:         Personal Use

📸 Photos:       7 available (cached)
   • First photo:   https://api.auto.dev/photos/retail/1C4AJWBGXHL638023-1.jpg...

🔗 Links:
   • VIN:           1C4AJWBGXHL638023
   • Listing:       https://www.carvana.com/vehicle/3898305...
   • CarFax:        https://www.carfax.com/VehicleHistory/p/Report.cfx?partner=CVN_0&vin=1...

#2. 2022 Jeep Cherokee Limited Edition
💰 Price:        $28,000
📏 Mileage:      23,815 miles
📍 Location:     Highl

## 5. Continue Conversation (Analytical Mode)

In [7]:
# Ask about a specific vehicle
response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "Tell me more about #1",
        "session_id": session_id
    }
)

data = response.json()

print(f"Mode: {data.get('mode')}")
print(f"\nAgent Response:")
print(data.get('response'))

Mode: None

Agent Response:
Here's a detailed look at the 2017 Jeep Wrangler Sahara you're interested in:

### Vehicle Details:
- **Make & Model:** Jeep Wrangler Sahara
- **Year:** 2017
- **Body Style:** Convertible SUV
- **Engine:** 6-cylinder Regular Unleaded
- **Transmission:** Manual
- **Drivetrain:** 4WD
- **Exterior Color:** Other
- **Interior Color:** Black
- **Seats:** 4
- **Doors:** 2
- **Fuel Type:** Regular unleaded

### Pricing & Dealer Information:
- **Price:** $28,990
- **Mileage:** 18,353 miles
- **Location:** Colorado Springs, CO
- **Dealer:** Carvana
- **Certified Pre-Owned:** No
- **Vehicle Listing Page:** [View on Carvana](https://www.carvana.com/vehicle/3898305)

### Vehicle History:
- **Accidents:** None reported
- **Previous Owners:** 1 (One-owner vehicle)
- **Usage:** Personal use

### Photos:
Here are some images of the vehicle:
1. ![Photo 1](https://api.auto.dev/photos/retail/1C4AJWBGXHL638023-1.jpg)
2. ![Photo 2](https://api.auto.dev/photos/retail/1C4AJWBGXHL6

## 6. Get Session State

In [8]:
response = requests.get(f"{BASE_URL}/session/{session_id}")
session_data = response.json()

print("Session State:")
print(f"  Filters: {session_data.get('filters')}")
print(f"  Preferences: {session_data.get('preferences')}")
print(f"  Vehicles: {len(session_data.get('vehicles', []))}")
print(f"  Conversation Length: {len(session_data.get('conversation_history', []))} messages")

Session State:
  Filters: {'make': 'Jeep', 'price': '28000-32000', 'state': 'CO'}
  Preferences: {'budget_sensitivity': 'moderate', 'brand_affinity': ['Jeep']}
  Vehicles: 10
  Conversation Length: 4 messages


## 7. View Conversation History

In [9]:
print("Conversation History:")
print("=" * 70)

for msg in session_data.get('conversation_history', []):
    role = msg.get('role', 'unknown').upper()
    content = msg.get('content', '')
    
    if role == 'USER':
        print(f"\n👤 User: {content}")
    else:
        print(f"\n🤖 Agent: {content[:200]}...")  # Truncate long responses
    print("-" * 70)

Conversation History:

👤 User: I want a Jeep around $30k in Colorado
----------------------------------------------------------------------

🤖 Agent: Hey there! I see you're on the hunt for a Jeep in Colorado within a $28-32k budget. There are a couple of interesting options here: one is priced at $28,000 with 23,815 miles, and another at $31,995 w...
----------------------------------------------------------------------

👤 User: Tell me more about #1
----------------------------------------------------------------------

🤖 Agent: Here's a detailed look at the 2017 Jeep Wrangler Sahara you're interested in:

### Vehicle Details:
- **Make & Model:** Jeep Wrangler Sahara
- **Year:** 2017
- **Body Style:** Convertible SUV
- **Engi...
----------------------------------------------------------------------


## 8. Get Vehicle Details by VIN

In [10]:
# Get VIN from first vehicle and test Auto.dev API directly
import os
from dotenv import load_dotenv

load_dotenv()

if vehicles:
    vin = vehicles[0].get('vehicle', {}).get('vin')
    
    if vin:
        print(f"Testing Auto.dev API for VIN: {vin}\n")
        print("=" * 70)
        
        # Get Auto.dev API key
        autodev_api_key = os.getenv('AUTODEV_API_KEY')
        
        if not autodev_api_key:
            print("❌ AUTODEV_API_KEY not found in environment")
        else:
            headers = {
                "Authorization": f"Bearer {autodev_api_key}",
                "Content-Type": "application/json"
            }
            
            # Get vehicle listing details
            print("\n1. Vehicle Listing Details:")
            listing_response = requests.get(
                f"https://api.auto.dev/listings/{vin}",
                headers=headers
            )
            
            if listing_response.status_code == 200:
                listing_data = listing_response.json()
                print("✅ Listing found")
                pprint(listing_data)
            else:
                print(f"❌ Error {listing_response.status_code}: {listing_response.text}")
            
            # Get vehicle photos
            print("\n2. Vehicle Photos:")
            photos_response = requests.get(
                f"https://api.auto.dev/photos/{vin}",
                headers=headers
            )
            
            if photos_response.status_code == 200:
                photos_data = photos_response.json()
                retail_photos = photos_data.get('data', {}).get('retail', [])
                
                print(f"✅ Found {len(retail_photos)} photos")
                
                if retail_photos:
                    print("\nFirst 5 photo URLs:")
                    for i, url in enumerate(retail_photos[:5], 1):
                        print(f"  {i}. {url}")
            else:
                print(f"❌ Error {photos_response.status_code}: {photos_response.text}")
    else:
        print("No VIN found in first vehicle")
else:
    print("No vehicles available to query")

Testing Auto.dev API for VIN: 1C4AJWBGXHL638023


1. Vehicle Listing Details:
✅ Listing found
{'actions': {'Toggle Link Domains': 'https://api.auto.dev/listings/1C4AJWBGXHL638023?domains=true'},
 'api': {'account': 'https://api.auto.dev/account',
         'description': '🚘 Vehicle Listings',
         'docs': 'https://docs.auto.dev/v2/products/vehicle-listings',
         'from': 'https://auto.dev',
         'name': 'api.auto.dev',
         'type': 'data',
         'url': 'https://api.auto.dev/listings'},
 'data': {'createdAt': '2025-09-20 17:19:52',
          'history': {'accidentCount': 0,
                      'accidents': False,
                      'oneOwner': True,
                      'ownerCount': 1,
                      'personalUse': True,
                      'usageType': 'Personal Use'},
          'location': [0, 0],
          'online': True,
          'retailListing': {'carfaxUrl': 'https://www.carfax.com/VehicleHistory/p/Report.cfx?partner=CVN_0&vin=1C4AJWBGXHL638023',


## 9. Custom Message (Try Your Own!)

In [11]:
# Try your own message
custom_message = "What's the safety rating of #1?"  

response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": custom_message,
        "session_id": session_id
    }
)

data = response.json()
print(f"Your Message: {custom_message}")
print(f"\nAgent Response:")
print(data.get('response'))

Your Message: What's the safety rating of #1?

Agent Response:
I couldn't find specific safety ratings for the 2017 Jeep Wrangler Sahara in the database. However, I can provide general information about the safety features and ratings for Jeep Wranglers from that era.

### General Safety Features for 2017 Jeep Wrangler:
- **Airbags:** Front-impact airbags for the driver and passenger, side-impact airbags, and overhead airbags.
- **Anti-lock Brakes (ABS):** Helps the vehicle maintain traction on slippery surfaces.
- **Stability Control:** Automatically senses when the vehicle's handling limits have been exceeded and reduces engine power and/or applies select brakes to help prevent the driver from losing control.
- **Pretensioners:** Automatically tighten seatbelts to place the occupant in the optimal seating position during a collision.
- **Security System:** The vehicle is equipped with a means of anticipating and/or detecting unwanted vehicle intrusion.

### General Safety Ratings:
Th

## 10. Reset Session

In [12]:
response = requests.post(
    f"{BASE_URL}/session/reset",
    json={"session_id": session_id}
)

print("Session reset:")
pprint(response.json())

Session reset:
{'session_id': 'a23406a0-f591-42c4-8b88-56e056acdd50', 'status': 'reset'}


## 11. List All Active Sessions

In [13]:
response = requests.get(f"{BASE_URL}/sessions")
sessions_data = response.json()

print(f"Active Sessions: {sessions_data.get('active_sessions')}")
print(f"Session IDs: {sessions_data.get('session_ids')}")

Active Sessions: 1
Session IDs: ['a23406a0-f591-42c4-8b88-56e056acdd50']


## 12. Test Event Tracking - Log User Interactions

In [14]:
# Simulate user interactions with the recommendation list

# Example 1: User views vehicle details
if vehicles and len(vehicles) > 0:
    first_vehicle = vehicles[0]
    vin = first_vehicle.get('vehicle', {}).get('vin')
    
    print("📝 Logging event: User clicked to view vehicle #1")
    response = requests.post(
        f"{BASE_URL}/session/{session_id}/event",
        json={
            "event_type": "vehicle_click",
            "data": {
                "vin": vin,
                "vehicle_index": 0,
                "year": first_vehicle.get('vehicle', {}).get('year'),
                "make": first_vehicle.get('vehicle', {}).get('make'),
                "model": first_vehicle.get('vehicle', {}).get('model')
            }
        }
    )
    
    if response.status_code == 200:
        result = response.json()
        print(f"✅ Event logged successfully!")
        print(f"   Event ID: {result.get('event_id')}")
        print(f"   Timestamp: {result.get('timestamp')}")
    else:
        print(f"❌ Error: {response.status_code} - {response.text}")

# Example 2: User views photos
print("\n📝 Logging event: User viewed photos")
response = requests.post(
    f"{BASE_URL}/session/{session_id}/event",
    json={
        "event_type": "photo_view",
        "data": {
            "vin": vin,
            "vehicle_index": 0,
            "photo_count": len(first_vehicle.get('photos', {}).get('retail', [])) if first_vehicle.get('photos') else 0
        }
    }
)

if response.status_code == 200:
    print(f"✅ Photo view event logged!")
    print(f"   Event ID: {response.json().get('event_id')}")

# Example 3: User clicks CarFax link
print("\n📝 Logging event: User clicked CarFax link")
response = requests.post(
    f"{BASE_URL}/session/{session_id}/event",
    json={
        "event_type": "link_click",
        "data": {
            "vin": vin,
            "link_type": "carfax",
            "url": first_vehicle.get('retailListing', {}).get('carfaxUrl')
        }
    }
)

if response.status_code == 200:
    print(f"✅ Link click event logged!")

# Example 4: Log a custom event (e.g., user scrolled to position 5)
print("\n📝 Logging event: User scrolled to vehicle #5")
response = requests.post(
    f"{BASE_URL}/session/{session_id}/event",
    json={
        "event_type": "scroll_position",
        "data": {
            "max_index_viewed": 4,
            "total_vehicles": len(vehicles)
        }
    }
)

if response.status_code == 200:
    print(f"✅ Scroll event logged!")

print("\n" + "="*70)

📝 Logging event: User clicked to view vehicle #1
✅ Event logged successfully!
   Event ID: 0
   Timestamp: 2025-10-14T11:07:04.054058

📝 Logging event: User viewed photos
✅ Photo view event logged!
   Event ID: 1

📝 Logging event: User clicked CarFax link
✅ Link click event logged!

📝 Logging event: User scrolled to vehicle #5
✅ Scroll event logged!



## 13. Retrieve All Logged Events

In [15]:
# Get all events for the session
print("🔍 Retrieving all interaction events...\n")

response = requests.get(f"{BASE_URL}/session/{session_id}/events")

if response.status_code == 200:
    events_data = response.json()
    
    print(f"📊 Total Events: {events_data.get('total')}")
    print(f"Session ID: {events_data.get('session_id')}\n")
    print("="*70)
    
    for i, event in enumerate(events_data.get('events', []), 1):
        print(f"\n📌 Event #{i}")
        print(f"   Type:      {event.get('event_type')}")
        print(f"   Timestamp: {event.get('timestamp')}")
        print(f"   Data:")
        
        for key, value in event.get('data', {}).items():
            print(f"      • {key}: {value}")
        
        print("-"*70)
    
    # Filter by event type
    print("\n\n🔍 Filtering: Only 'vehicle_click' events\n")
    response = requests.get(f"{BASE_URL}/session/{session_id}/events?event_type=vehicle_click")
    
    if response.status_code == 200:
        filtered_data = response.json()
        print(f"Found {filtered_data.get('total')} vehicle click events")
        for event in filtered_data.get('events', []):
            vin = event.get('data', {}).get('vin')
            print(f"   • VIN: {vin} at {event.get('timestamp')}")
else:
    print(f"❌ Error: {response.status_code} - {response.text}")

🔍 Retrieving all interaction events...

📊 Total Events: 4
Session ID: a23406a0-f591-42c4-8b88-56e056acdd50


📌 Event #1
   Type:      vehicle_click
   Timestamp: 2025-10-14T11:07:04.054058
   Data:
      • vin: 1C4AJWBGXHL638023
      • vehicle_index: 0
      • year: 2017
      • make: Jeep
      • model: Wrangler
----------------------------------------------------------------------

📌 Event #2
   Type:      photo_view
   Timestamp: 2025-10-14T11:07:04.058312
   Data:
      • vin: 1C4AJWBGXHL638023
      • vehicle_index: 0
      • photo_count: 7
----------------------------------------------------------------------

📌 Event #3
   Type:      link_click
   Timestamp: 2025-10-14T11:07:04.062494
   Data:
      • vin: 1C4AJWBGXHL638023
      • link_type: carfax
      • url: https://www.carfax.com/VehicleHistory/p/Report.cfx?partner=CVN_0&vin=1C4AJWBGXHL638023
----------------------------------------------------------------------

📌 Event #4
   Type:      scroll_position
   Timestamp: 2025-